# Simple DML

In [15]:
import numpy as np
from doubleml import DoubleMLData, DoubleMLPLR
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

import numpy as np

SEED = 42
N_SAMPLES = 1000
N_FEATURES = 10

np.random.seed(SEED)

X = np.random.normal(0, 1, size=(N_SAMPLES, N_FEATURES))
treatment_prob = 1 / (1 + np.exp(-0.5 * X[:, 0]))
t = np.random.binomial(1, treatment_prob, size=N_SAMPLES)
y = 2 * t + 0.3 * X[:, 1] - 0.5 * X[:, 2] + np.random.normal(0, 1, size=N_SAMPLES)

data = DoubleMLData.from_arrays(X, y, t)

# Outcome model m(X)
model_m_x = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)

# Propensity score model g(X)
model_g_x = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)

dml_plr = DoubleMLPLR(data, model_m_x, model_g_x, n_folds=5)

dml_plr.fit()

ate = dml_plr._coef[0]
se = dml_plr._se[0]
print(f"ATE: {ate:.4f}; SE: {se:.4f}")

ATE: 1.8844; SE: 0.0643


# Simple DML with Hyperparameter Tuning

In [26]:
from sklearn.model_selection import GridSearchCV

param_grid_m = {'max_depth': [3, 5, 7]}
param_grid_g = {'max_depth': [3, 5, 7]}

data = DoubleMLData.from_arrays(X, y, t)

base_learner_m = RandomForestRegressor(n_estimators=100, random_state=42)
ml_m = GridSearchCV(base_learner_m, param_grid_m, cv=3)

base_learner_g = RandomForestClassifier(n_estimators=100, random_state=42)
ml_g = GridSearchCV(base_learner_g, param_grid_g, cv=3)

dml_plr = DoubleMLPLR(data, ml_m, ml_g, n_folds=5)
dml_plr.fit()

ate = dml_plr._coef[0]
se = dml_plr._se[0]
print(f"ATE: {ate:.4f}; SE: {se:.4f}")
dml_plr.summary

ATE: 1.8962; SE: 0.0661


,coef,std err,t,P>|t|,2.5 %,97.5 %
d,1.896212,0.06611,28.682832,6.247396e-181,1.76664,2.025785


# Use Different Covariates to Predict Treatment and Target

In [45]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

import numpy as np
import pandas as pd

n_samples = 1000

# Covariates
X1 = np.random.normal(size=n_samples)
X2 = np.random.normal(size=n_samples)
X3 = np.random.normal(size=n_samples)
X4 = np.random.normal(size=n_samples)
X5 = np.random.normal(size=n_samples)
X6 = np.random.normal(size=n_samples)

# DataFrame
df = pd.DataFrame({
    'X1': X1,
    'X2': X2,
    'X3': X3,
    'X4': X4,
    'X5': X5,
    'X6': X6,
})

covariates_g = [0, 1, 2]
covariates_m = [1, 3, 5]

from scipy.special import expit

df['treatment'] = np.random.binomial(1, expit(0.5 * df['X1'] - 0.3 * df['X2'] + 0.2 * df['X3']))
df['outcome'] = 2 * df['treatment'] + 0.5 * df['X4'] - 0.7 * df['X5'] + 0.3 * df['X6'] + np.random.normal(size=n_samples)

# For g(X)
preprocessor_g = ColumnTransformer(
    transformers=[
        ('select_columns', 'passthrough', covariates_g)
    ],
    remainder='drop'
)

pipeline_g = Pipeline([
    ('preprocessor', preprocessor_g),
    ('model', RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42))
])

# For m(X)
preprocessor_m = ColumnTransformer(
    transformers=[
        ('select_columns', 'passthrough', covariates_m)
    ],
    remainder='drop'
)

pipeline_m = Pipeline([
    ('preprocessor', preprocessor_m),
    ('model', RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42))
])

dml_plr = DoubleMLPLR(data, pipeline_m, pipeline_g, n_folds=5)
dml_plr.fit()

dml_plr.summary

,coef,std err,t,P>|t|,2.5 %,97.5 %
d,1.937841,0.07312,26.502299,9.118737e-155,1.794529,2.081153


# Do GridSearchCV with Pipeline

In [53]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Preprocessing step: select relevant columns
preprocessor_m = ColumnTransformer(
    transformers=[
        ('select_columns', 'passthrough', [1, 2, 3])
    ],
    remainder='drop'
)

pipeline_m = Pipeline([
    ('preprocessor', preprocessor_m),
    ('model', RandomForestRegressor(random_state=42))
])

param_grid_m = {
    'model__n_estimators': [50, 100, 200],
    'model__max_depth': [3, 5, 10]
}

grid_search_m = GridSearchCV(
    estimator=pipeline_m,
    param_grid=param_grid_m,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

from doubleml import DoubleMLData, DoubleMLPLR

data = DoubleMLData(df, y_col='outcome', d_cols='treatment')

pipeline_g = Pipeline([
    ('preprocessor', ColumnTransformer(
        transformers=[('select_columns', 'passthrough', [1, 2, 3])],
        remainder='drop'
    )),
    ('model', RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42))
])

param_grid_g = {
    'model__n_estimators': [50, 100, 200],
    'model__max_depth': [3, 5, 10]
}

grid_search_g = GridSearchCV(
    estimator=pipeline_g,
    param_grid=param_grid_g,
    cv=5,
    scoring='balanced_accuracy',
    n_jobs=-1
)

dml_plr = DoubleMLPLR(data, grid_search_m, grid_search_g, n_folds=5)

dml_plr.fit()
dml_plr.summary

,coef,std err,t,P>|t|,2.5 %,97.5 %
treatment,2.009888,0.079271,25.354521,8.011809e-142,1.854519,2.165257


In [55]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.compose import ColumnTransformer

# Preprocessing step: select relevant columns and create polynomial features
preprocessor_lasso = ColumnTransformer(
    transformers=[
        ('select_columns', 'passthrough', [1, 2, 3])  # Columns to use
    ],
    remainder='drop'
)

lasso_pipeline = Pipeline([
    ('preprocessor', preprocessor_lasso),
    ('poly_features', PolynomialFeatures(degree=2, include_bias=False)),  # Add polynomial features
    ('scaler', StandardScaler()),  # Standardize features
    ('lasso', LassoCV(cv=5, random_state=42))  # Cross-validated LASSO
])

In [56]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV

# Preprocessing step: select relevant columns
preprocessor_dnn = ColumnTransformer(
    transformers=[
        ('select_columns', 'passthrough', [1, 2, 3])  # Columns to use
    ],
    remainder='drop'
)

dnn_pipeline = Pipeline([
    ('preprocessor', preprocessor_dnn),
    ('scaler', StandardScaler()),  # Standardize features
    ('mlp', MLPRegressor(max_iter=1000, random_state=42))  # Neural network
])

# Define the parameter grid for the DNN
param_grid_dnn = {
    'mlp__hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],  # Architectures
    'mlp__alpha': [0.001, 0.01, 0.1, 1],  # Ridge regularization
    'mlp__activation': ['relu', 'tanh'],  # Activation functions
    'mlp__learning_rate': ['constant', 'adaptive']  # Learning rate strategies
}

# Wrap the pipeline in GridSearchCV
grid_search_dnn = GridSearchCV(
    estimator=dnn_pipeline,
    param_grid=param_grid_dnn,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

# Instrumental Variable

In [67]:
import numpy as np
import pandas as pd
from doubleml import DoubleMLPLIV, DoubleMLData
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

def get_column_indices(dataframe: pd.DataFrame, column_names: list) -> list[int]:
    """
    Get the column indices of specified column names in a pandas DataFrame.

    Parameters:
    dataframe (pd.DataFrame): Theb DataFrame containing the columns.
    column_names (list): A list of column names for which to retrieve indices.

    Returns:
    list: A list of column indices corresponding to the specified column names.
    """
    # Ensure all column names are present in the DataFrame
    missing_columns = [col for col in column_names if col not in dataframe.columns]
    if missing_columns:
        raise ValueError(f"The following columns are not in the DataFrame: {missing_columns}")
    
    return [dataframe.columns.get_loc(col) for col in column_names]

# Generate synthetic data
np.random.seed(42)
n_samples = 1000
n_features = 5

X = np.random.normal(size=(n_samples, n_features))
Z = np.random.binomial(1, 0.5, size=n_samples)  # Binary instrument
D = 0.5 * Z + 0.3 * X[:, 0] + np.random.normal(0, 1, size=n_samples)
Y = 2.0 * D + 0.5 * X[:, 1] + np.random.normal(0, 1, size=n_samples)

# Create DataFrame
df = pd.DataFrame(X, columns=[f'X{i+1}' for i in range(n_features)])
df['Z'] = Z
df['D'] = D
df['Y'] = Y

# Specify DoubleML data
data = DoubleMLData(df, y_col='Y', d_cols='D', z_cols='Z')

x_indices = get_column_indices(df, x_columns)

# Define a pipeline for outcome model m(X)
preprocessor_m = ColumnTransformer(
    transformers=[('scaler', StandardScaler(), x_indices)],  # Use indices
    remainder='drop'
)

pipeline_m = Pipeline([
    ('preprocessor', preprocessor_m),
    ('model', RandomForestRegressor(random_state=42))
])

param_grid_m = {
    'model__n_estimators': [50, 100, 200],
    'model__max_depth': [3, 5, 10]
}

grid_search_m = GridSearchCV(
    estimator=pipeline_m,
    param_grid=param_grid_m,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

# Define a pipeline for treatment model l(Z, X)
preprocessor_l = ColumnTransformer(
    transformers=[('scaler', StandardScaler(), x_indices)],  # Indices for X and Z
    remainder='drop'
)

pipeline_l = Pipeline([
    ('preprocessor', preprocessor_l),
    ('model', RandomForestRegressor(random_state=42))
])

param_grid_l = {
    'model__n_estimators': [50, 100, 200],
    'model__max_depth': [3, 5, 10]
}

grid_search_l = GridSearchCV(
    estimator=pipeline_l,
    param_grid=param_grid_l,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

# Define a pipeline for instrument model r(Z, X)
pipeline_r = Pipeline([
    ('preprocessor', preprocessor_l),  # Similar preprocessor as l(Z, X)
    ('model', RandomForestRegressor(random_state=42))
])

param_grid_r = {
    'model__n_estimators': [50, 100, 200],
    'model__max_depth': [3, 5, 10]
}

grid_search_r = GridSearchCV(
    estimator=pipeline_r,
    param_grid=param_grid_r,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

# Initialize DoubleMLPLIV object
dml_pliv = DoubleMLPLIV(
    obj_dml_data=data,
    ml_l=grid_search_l,  # Treatment model
    ml_m=grid_search_m,  # Outcome model
    ml_r=grid_search_r   # Instrument model
)

# Fit the model
dml_pliv.fit()

# Print summary
print(dml_pliv.summary)

       coef   std err          t         P>|t|     2.5 %    97.5 %
D  1.990362  0.114688  17.354629  1.819723e-67  1.765579  2.215146


# Frontdoor Adjustment

In [207]:
import numpy as np
import pandas as pd
from doubleml import DoubleMLPLIV, DoubleMLData
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

# Generate synthetic data for front-door adjustment
np.random.seed(42)
for _ in range(5):
    n_samples = 100
    X = np.random.normal(size=(n_samples, 3))  # Covariates
    D = np.random.binomial(1, 0.5, size=n_samples)  # Treatment
    M = 0.8 * D + 0.2 * X[:, 0] + np.random.normal(0, 0.1, size=n_samples)  # Mediator
    Y = 1.5 * M + 0.2 * X[:, 0] + np.random.normal(0, 0.1, size=n_samples)  # Outcome

    df = pd.DataFrame(X, columns=['X1', 'X2', 'X3'])
    df['D'] = D
    df['M'] = M
    df['Y'] = Y

    # DoubleML data
    data = DoubleMLData(df, y_col='Y', d_cols='D', z_cols='M')

    # Define learners
    learner = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)

    # Models for nuisance functions
    ml_l = GridSearchCV(Pipeline([('model', learner)]), param_grid={}, cv=3)
    ml_m = GridSearchCV(Pipeline([('model', learner)]), param_grid={}, cv=3)
    ml_r = GridSearchCV(Pipeline([('model', learner)]), param_grid={}, cv=3)

    # DoubleMLPLIV object
    dml_pliv = DoubleMLPLIV(data, ml_l=ml_l, ml_m=ml_m, ml_r=ml_r)

    # Fit the model
    dml_pliv.fit()

    # Print the summary
    print(dml_pliv.summary)

       coef   std err          t          P>|t|     2.5 %   97.5 %
D  1.260351  0.046837  26.909214  1.713417e-159  1.168552  1.35215
       coef   std err          t          P>|t|     2.5 %   97.5 %
D  1.246652  0.052709  23.651739  1.132662e-123  1.143345  1.34996
       coef   std err          t          P>|t|     2.5 %    97.5 %
D  1.295172  0.048491  26.709593  3.641799e-157  1.200131  1.390212
       coef   std err        t          P>|t|     2.5 %    97.5 %
D  1.227967  0.039574  31.0295  2.157074e-211  1.150403  1.305531
       coef  std err          t          P>|t|     2.5 %    97.5 %
D  1.347179  0.04646  28.996437  7.296705e-185  1.256119  1.438239


# Correlation Between Features

In [205]:
from sklearn.datasets import make_sparse_spd_matrix, make_spd_matrix

d = 10
avg_corr_covariates = .1

cov_matrix = make_sparse_spd_matrix(d, alpha=0) * make_spd_matrix(d)
pd.DataFrame(cov_matrix)

,0,1,2,3,4,5,6,7,8,9
0,6.201532,0.043825,0.297731,-2.514653,0.102072,0.087034,-0.401188,1.952198,-0.381415,0.478923
1,0.043825,0.767785,-0.113593,0.151767,0.032833,-0.000447,0.003809,-0.003126,-0.034657,0.019952
2,0.297731,-0.113593,7.877645,-0.183830,0.063273,0.017774,-0.416667,0.289051,-0.414976,-0.382765
3,-2.514653,0.151767,-0.183830,9.472169,0.163902,-0.167743,0.697450,-1.314637,0.416422,-0.271861
4,0.102072,0.032833,0.063273,0.163902,1.429548,0.019877,-0.024886,0.170254,0.078595,0.043645
5,0.087034,-0.000447,0.017774,-0.167743,0.019877,1.365512,-0.100100,0.067493,-0.032747,0.005909
6,-0.401188,0.003809,-0.416667,0.697450,-0.024886,-0.100100,2.303199,0.105149,0.192926,-0.021746
7,1.952198,-0.003126,0.289051,-1.314637,0.170254,0.067493,0.105149,6.960944,-0.137170,0.304518
8,-0.381415,-0.034657,-0.414976,0.416422,0.078595,-0.032747,0.192926,-0.137170,0.789918,-0.054666
9,0.478923,0.019952,-0.382765,-0.271861,0.043645,0.005909,-0.021746,0.304518,-0.054666,2.181609
